In [ ]:
import pandas as pd 
postcodes = pd.read_csv("postcodes_close.csv")

In [ ]:
pc_to_scrape = list(postcodes["PCDS"])

In [ ]:
#http://landregistry.data.gov.uk/app/root/doc/ppd
sparql = """
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix owl: <http://www.w3.org/2002/07/owl#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>
prefix sr: <http://data.ordnancesurvey.co.uk/ontology/spatialrelations/>
prefix ukhpi: <http://landregistry.data.gov.uk/def/ukhpi/>
prefix lrppi: <http://landregistry.data.gov.uk/def/ppi/>
prefix skos: <http://www.w3.org/2004/02/skos/core#>
prefix lrcommon: <http://landregistry.data.gov.uk/def/common/>

# Returns the Price Paid data from the default graph for each transaction record having
# an address with the given postcode.
# The postcode to query is set using SPARQL 1.1's 'values' clause

SELECT ?paon ?saon ?street ?town ?county ?postcode ?amount ?date ?category ?propertyType
WHERE
{{
  VALUES ?postcode {{"{}"^^xsd:string}}

  ?addr lrcommon:postcode ?postcode.

  ?transx lrppi:propertyAddress ?addr ;
          lrppi:pricePaid ?amount ;
          lrppi:transactionDate ?date ;
          lrppi:propertyType ?propertyType ;
          lrppi:transactionCategory/skos:prefLabel ?category.

  OPTIONAL {{?addr lrcommon:county ?county}}
  OPTIONAL {{?addr lrcommon:paon ?paon}}
  OPTIONAL {{?addr lrcommon:saon ?saon}}
  OPTIONAL {{?addr lrcommon:street ?street}}
  OPTIONAL {{?addr lrcommon:town ?town}}
}}
ORDER BY ?amount
"""

In [ ]:
url = "http://landregistry.data.gov.uk/landregistry/query"

In [ ]:
import requests
import json

In [ ]:
counter = 0
results_list = []
for pc in pc_to_scrape:
    
    if counter % 50 == 0:
        print(f"Percentage complete is {counter/len(postcodes):0.1%}.  Counter is: {counter}. Number of postcodes is {len(results_list):,.0f}.")
    counter += 1
    this_sparql = sparql.format(pc)
    payload = {"query": this_sparql}
    res = requests.post(url, payload)
    results = json.loads(res.text)
    bindings = results['results']['bindings']

    for_pd = []
    for result in bindings:
        new_obj = {}
        for key in result:
            new_obj[key] = result[key]['value']
        for_pd.append(new_obj)
#     print(f"There were {len(for_pd)} sales found in {pc}")
    
    results_list.extend(for_pd)

In [ ]:
import pandas as pd 
df = pd.DataFrame(results_list)


In [ ]:
len(df)

In [ ]:
f1 = df["category"] == "Standard price paid transaction"
df = df[f1]
df["saon"] = df["saon"].fillna("")
df["paon"] = df["paon"].fillna("")
df["address"] = df["paon"] + " " + df["saon"] + " " + df["street"] + " " + df["town"] + " " + df["postcode"]
df["propertyType"] = df["propertyType"].str.replace("http://landregistry.data.gov.uk/def/common/", "")
df.head(2)

In [ ]:
cols = ["amount", "address", "propertyType", "date", "postcode"]
df_cols = df[cols]
df_cols.head()

In [ ]:
df_merged = df_cols.merge(postcodes, left_on="postcode", right_on="PCDS")

In [ ]:
cols.extend(["LAT", "LONG", "dist_from_home_km", "dist_from_kl_km"])
df_merged_cols = df_merged[cols]

In [ ]:
df_merged_cols.to_csv("all_.csv")

In [ ]:
import altair as alt

In [ ]:
df["date"] = pd.to_datetime(df["date"])
df.to_csv("one_and_half_kings_langley.csv", index=False)

In [ ]:
alt.Chart(df.sample(5000, replace=False), width=1000).mark_bar().encode(x="yearmonth(date):T",y='count()')

In [ ]:
# # New flats which sold 
# f1 = df["date"].dt.month == 5
# f2 = df["date"].dt.year == 2017
# f3 = df["street"].str.contains("WOODS")
# df_f = df[f1 & f2 & f3]
# df_f.sort_values("saon")